In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/navar/NAVAR

Mounted at /content/drive
/content/drive/MyDrive/navar/NAVAR


In [ ]:
!pip install -r requirements.txt

In [3]:
from train_NAVAR import train_NAVAR
from evaluate import calculate_AUROC, dream_file_to_causal_matrix
import pandas as pd
import argparse

In [4]:
experiment = 'ecoli1' # experiment name
# should accuracy be evaluated?
evaluate = True
# should we use LSTM?
lstm = True 
lambda1 = 0.26025947107502856
batch_size = 46 # batch size for training
wd = 1.4961159190152877e-05
hidden_nodes = 10
learning_rate = 0.002 
hl = 1 # number of hidden layers

In [10]:
# load the data
file = f'experiments/{experiment}.tsv'
ground_truth_file = f'experiments/{experiment}_gt.txt'
data = pd.read_csv(file, sep='\t')
data = data.values[:, 1:]
epochs = 100
maxlags = 21 if lstm else 2

In [11]:
# start training
print(f"Starting training on the data from experiment {experiment}, training for {epochs} iterations.")
score_matrix, _, _ = train_NAVAR(data, maxlags=maxlags, hidden_nodes=hidden_nodes, dropout=0, epochs=epochs,
                                 learning_rate=learning_rate, batch_size=batch_size, lambda1=lambda1,
                                 val_proportion=0.0, weight_decay=wd, check_every=500, hidden_layers=hl, normalize=True,
                                 split_timeseries=21, lstm=lstm)
# evaluate
print('Done training!')
if evaluate:
    ground_truth_matrix = dream_file_to_causal_matrix(ground_truth_file)
    AUROC = calculate_AUROC(score_matrix, ground_truth_matrix, ignore_self_links=True)
    print(f"The AUROC of this model on experiment {experiment} is: {AUROC}")

Starting training on the data from experiment ecoli1, training for 100 iterations.
Done training!
The AUROC of this model on experiment ecoli1 is: 0.5387457800511508
